### Importing Libraries

In [4]:
from Projects.OLIST_SALES.config.config import get_environment_info,get_today_date,check_for_duplicates
from pyspark.sql.functions import col,when,count

### Read Config File

In [5]:
storage_info = get_environment_info()
source_name = "olist"
storage_account = storage_info[0]
landing_container = storage_info[1]
raw_container = storage_info[2]
load_date = get_today_date()
print(load_date)

2024-12-07


#### Connect to ADLS using SAS Token

### Data Quality Checks

#### read the data

In [6]:
customer_df = read_csv_file(path=f"abfss://{landing_container}@{storage_account}.dfs.core.windows.net/olist_customers_dataset.csv")
geolocation_df = read_csv_file(path=f"abfss://{landing_container}@{storage_account}.dfs.core.windows.net/olist_geolocation_dataset.csv")
orders_df = read_csv_file(path=f"abfss://{landing_container}@{storage_account}.dfs.core.windows.net/olist_orders_dataset.csv")
orders_items_df = read_csv_file(path=f"abfss://{landing_container}@{storage_account}.dfs.core.windows.net/olist_order_items_dataset.csv")
products_df = read_csv_file(path=f"abfss://{landing_container}@{storage_account}.dfs.core.windows.net/olist_products_dataset.csv")
products_english_df = read_csv_file(path=f"abfss://{landing_container}@{storage_account}.dfs.core.windows.net/product_category_name_translation.csv")
orders_payment_df = read_csv_file(path=f"abfss://{landing_container}@{storage_account}.dfs.core.windows.net/olist_order_payments_dataset.csv")
orders_review_df = read_csv_file(path=f"abfss://{landing_container}@{storage_account}.dfs.core.windows.net/olist_order_reviews_dataset.csv")

NameError: name 'spark' is not defined

#### Check the null counts

In [ ]:
datasets = {
    "customer_df": customer_df,
    "orders_df": orders_df,
    "orders_items_df": orders_items_df,
    "products_df": products_df,
    "orders_payment_df": orders_payment_df,
    "orders_review_df": orders_review_df,
}

for df_name, dataframe in datasets.items():
    columns_to_select = []
    
    # Create null and not-null counts for each column
    for c in dataframe.columns:
        columns_to_select.append(count(when(col(c).isNull(), c)).alias(f"{c}_null_count"))
        columns_to_select.append(count(when(col(c).isNotNull(), c)).alias(f"{c}_not_null_count"))
    
    # Calculate results
    result = dataframe.select(*columns_to_select)
    
    # Write the result to CSV
    result.coalesce(1).write.format("csv") \
        .option("header", True) \
        .option("delimiter", ",") \
        .save(f"abfss://{raw_container}@{storage_account}.dfs.core.windows.net/null_or_not_null/{load_date}/{df_name}")

#### Check If Data Contains Duplicate Records

In [ ]:
customer_df_columns = ['customer_id','customer_unique_id']
orders_df_columns = ['order_id']
products_df_columns = ['product_id']
        
# Check for duplicates in each DataFrame
try:
    check_for_duplicates(customer_df, customer_df_columns, 'customer_df')
    check_for_duplicates(orders_df, orders_df_columns, 'orders_df')
    check_for_duplicates(products_df, products_df_columns, 'products_df')
    print("No duplicates found in the specified columns.")
except Exception as e:
    print(str(e))
    dbutils.notebook.exit("Notebook failed due to duplicates.")


#### Copy Data from Landing to Raw

In [9]:
datasets = {
    "customer": customer_df,
    "orders": orders_df,
    "orders_items": orders_items_df,
    "products": products_df,
    "orders_payment": orders_payment_df,
    "orders_review": orders_review_df
}

for dataset,dataframe in datasets.items():
    dataframe.coalesce(1).write.format("parquet") \
        .option("header", True) \
        .option("delimiter", ",") \
        .save(f"abfss://{raw_container}@{storage_account}.dfs.core.windows.net/null_or_not_null/{load_date}/{dataset}/")

customer_df customer_df
